Método de Newton para solución de sistemas de ecuaciones no lineales

In [5]:
import numpy as np
from sympy.abc import x
from sympy import *
import copy
import math as m


class GaussJordan:

    def intercambiarFilas(self, index1, index2, M): 
        M[index1],M[index2]=M[index2],M[index1]
        return M
   
    def multiplicarFila(self, k, fila, colInicial, M):
        M[fila] = k * M[fila]
        return M

    def restarFilas(self, f1, f2, M):
        M[f1] =  M[f2] - M[f1]
        return M 

    def buscarPivote(self, filas, col, M):
        indiceFila = -1
        maxNum = np.inf *-1
        for i in range(col+1, filas):
            if(M[i][col] > maxNum and M[i][col] != 0):
                indiceFila = i
                maxNum = M[i][col]
        return indiceFila

    def eliminacionGaussiana(self, f, c, M):
        # Definición de variables
        indicePiv = -1
        
        for i in range(f):
            pivote = M[i][i]
            if pivote == 0:
                indicePiv = self.buscarPivote(f, i, M) # Se implementa pivoteo parcial
                #TODO: Implementar pivoteo completo
                if indicePiv == -1:
                    print("El sistema no tiene solución")
                    exit(0)
                else:
                    M = self.intercambiarFilas(indicePiv, i, M)
                    pivote = M[i][i]
            
            for j in range(i+1, f): # Realizar la eliminación de los elementos debajo del pivote
                if M[j][i] != 0:
                    k = pivote / M[j][i]    # Multiplicador para la eliminación
                    M = self.multiplicarFila(k, j, i, M)
                    M = self.restarFilas(j, i, M)

    def GJ(self,f,c,M):
        for i in range(f):
            pivote=M[f-1-i][f-1-i]
            M[f-1-i]=M[f-1-i]/pivote
            for j in range(f-1-i):
              M[j]=M[j]-M[j][f-1-i]*M[f-1-i]

    def inversa(self,f,M):
      I=np.identity(f)
      matriz=np.concatenate((M,I),axis=1)
      return matriz

    def obtenerInversa(self,f,matriz):
      inversa=[]
      for i in range(f):
        aux=[]
        for j in range(f):
          aux.append(matriz[i][j+len(matriz)])
        inversa.append(aux)
      return inversa

class NewtonMethod:

  def jacobian(self,xs,initPoint,F):
    J=[] # Creación de Jacobiana
    for i in range(len(xs)):
      aux=[]
      for j in range(len(xs)):
        aux.append(diff(F[i],xs[j]))
      J.append(aux)

    # Evaluar en Jacobiana
    for i in range(len(J)):
      for j in range(len(J)):
        for k in range(len(initPoint)):
          J[i][j]=J[i][j].subs(xs[k],initPoint[k])
          
    return J

  def eqnewton(self,xs,initPoint,Jinversa,F):
    # Evaluar en F(x)
    F1=copy.copy(F)
    for i in range(len(F1)):
      for j in range(len(initPoint)):
        F1[i]=F1[i].subs(xs[j],initPoint[j])

    initPoint=initPoint-np.dot(Jinversa,F1)

    return initPoint

  def norma(self,vector0,vector1):
    # Cálculo de error por definición de norma
    suma=0
    for i in range(len(vector0)):
      aux=(vector0[i]-vector1[i])**2
      suma=suma+aux
    return m.sqrt(suma)
    
  def newton(self,xs,F,initPoint):
    tol=0.0001
    maxIter=100
    iteracion=0
    error=np.inf
    objN=NewtonMethod()
    objGJ=GaussJordan()

    while (error>=tol or iteracion==maxIter):
      aproximacion=initPoint
      J=objN.jacobian(xs,aproximacion,F)

      matriz=objGJ.inversa(len(J),J)
      objGJ.eliminacionGaussiana(len(J),len(J[0]),matriz)
      objGJ.GJ(len(J),len(J[0]),matriz)

      inversa=objGJ.obtenerInversa(len(J),matriz)

      solucion=objN.eqnewton(xs,aproximacion,inversa,F)

      # Cálculo de error
      error=objN.norma(solucion,aproximacion)
      initPoint=solucion
      print("iteracion",iteracion,"solucion",solucion)
      iteracion=iteracion+1

    return initPoint


def main():
  n=3 # Número de ecuaciones y variables
  xs=[] # Incógnitas
  for i in range(n):
    xs.append(symbols("x"+str(i+1)))

  # Funciones
  #f1=xs[0]**2-xs[1]**2-1
  #f2=xs[0]**2+xs[1]**2+xs[0]*xs[1]-4
  f1=3*xs[0] - cos(xs[1]*xs[2]) - 1/2
  f2=xs[0]**2 - 81*(xs[1] + 0.1)**2 + sin(xs[2]) + 1.06
  f3=np.e**(-xs[0]*xs[1]) + 20*xs[2] + ((10 * np.pi) -3) /3
  F=[]
  F.append(f1)
  F.append(f2)
  F.append(f3)

  # Punto inicial
  #initPoint=[1,1]
  initPoint=[0.1,0.1,-0.1]
  print("Punto inicial:",initPoint)
  
  objNM=NewtonMethod()
  aproximation=objNM.newton(xs,F,initPoint)

  print("Aproximación final:",aproximation)

if __name__=="__main__":
  main()

Punto inicial: [0.1, 0.1, -0.1]
iteracion 0 solucion [0.499869672926428 0.0194668485374181 -0.521520471935831]
iteracion 1 solucion [0.500014240164219 0.00158859137029390 -0.523556964347638]
iteracion 2 solucion [0.500000113467834 1.24447833215536e-5 -0.523598450072889]
iteracion 3 solucion [0.500000000007076 7.75785726225503e-10 -0.523598775578007]
Aproximación final: [0.500000000007076 7.75785726225503e-10 -0.523598775578007]
